In [54]:
import numpy as np
import pandas as pd

In [55]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [56]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA2_K/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [57]:
mineralogy = mineralogy.dropna()

In [58]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [59]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [60]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [61]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6769,72.811181,0.254636,14.705736,0.845249,1.896637,0.060299,0.567699,1.307029,3.509223,4.042311
6770,72.804069,0.254979,14.708530,0.845526,1.898604,0.060344,0.568851,1.309250,3.509013,4.040833
6771,72.797301,0.255305,14.711185,0.845790,1.900476,0.060387,0.569948,1.311366,3.508814,4.039428
6772,72.790960,0.255611,14.713670,0.846037,1.902229,0.060427,0.570977,1.313349,3.508626,4.038113
6773,72.785124,0.255893,14.715955,0.846264,1.903843,0.060464,0.571925,1.315176,3.508454,4.036904


In [62]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6769,72.811181,0.254636,14.705736,0.845249,1.896637,0.060299,0.567699,1.307029,3.509223,4.042311
6770,72.804069,0.254979,14.708530,0.845526,1.898604,0.060344,0.568851,1.309250,3.509013,4.040833
6771,72.797301,0.255305,14.711185,0.845790,1.900476,0.060387,0.569948,1.311366,3.508814,4.039428
6772,72.790960,0.255611,14.713670,0.846037,1.902229,0.060427,0.570977,1.313349,3.508626,4.038113
6773,72.785124,0.255893,14.715955,0.846264,1.903843,0.060464,0.571925,1.315176,3.508454,4.036904
...,...,...,...,...,...,...,...,...,...,...
265168,67.609120,0.522335,15.991789,0.991590,3.282887,0.088957,1.692748,3.253772,3.326965,3.239837
265169,67.611197,0.522224,15.991507,0.991550,3.282362,0.088947,1.692195,3.252890,3.327046,3.240082
265170,67.612790,0.522139,15.991290,0.991519,3.281959,0.088939,1.691771,3.252213,3.327108,3.240271
265171,67.613878,0.522081,15.991142,0.991498,3.281684,0.088934,1.691482,3.251751,3.327151,3.240399


In [63]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:95000]
mineralogy3 = mineralogy.iloc[95000:125000]

In [64]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA2_K/area2_K_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA2_K/area2_K_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA2_K/area2_K_3.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [65]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2_K/area2_K_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2_K/area2_K_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2_K/area2_K_3_results_excel.txt")

In [66]:
df_final = pd.concat([df_final1, df_final2, df_final3])

In [67]:
df_final

,Q,P,K
SAMPLE,,,
6769,32.151649,33.206601,26.855727
6770,32.143635,33.216016,26.846803
6771,32.135941,33.225010,26.838361
6772,32.128750,33.233461,26.830406
6773,32.122177,33.241168,26.823107
...,...,...,...
265168,24.305342,41.476023,21.959614
265169,24.309034,41.472342,21.961109
265170,24.311922,41.469426,21.962281


## QAPF classification

In [68]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [69]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro',
       'monzodiorite monzogabbro'], dtype=object)

In [70]:
df_final.to_excel("../_CIPW/CIPW/AREA2_K/qapf.xlsx", index=True)

-------

In [71]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA2_K/QAPF_counts.xlsx")

-----

In [72]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area2_K.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [73]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,788062.538051,4.936585e+06,53.0,T,2.0,52.526539,17.979983,22.933498,monzo granite
7,609204.390926,5.261750e+06,53.0,T,2.0,43.913578,28.900560,23.948371,monzo granite
8,683161.686140,5.345172e+06,53.0,T,2.0,55.002583,18.817920,19.755253,monzo granite
13,566277.769165,5.115580e+06,53.0,T,2.0,42.445293,29.966574,23.523890,monzo granite
16,513561.171031,5.820370e+06,53.0,T,2.0,40.720371,33.121540,20.140040,monzo granite
...,...,...,...,...,...,...,...,...,...
4638,746609.482852,5.293856e+06,53.0,T,2.0,6.645143,62.556135,9.683687,quartz monzodiorite\nquartz monzogabbro
4642,577326.668959,5.042370e+06,53.0,T,2.0,3.695705,49.300256,19.926339,quartz monzodiorite\nquartz monzogabbro
4649,461509.372701,5.662951e+06,53.0,T,2.0,3.945260,59.519575,6.028671,quartz diorite\nquartz gabbro\nquartz anorthosite
4654,620259.749413,5.148324e+06,53.0,T,2.0,0.000000,52.913423,17.411244,monzodiorite monzogabbro


In [74]:
QAPF_control_area2_K = QAPF_control[QAPF_control['area'] ==2]
QAPF_control_area2_K.to_excel("../_CIPW/CIPW/AREA2_K/QAPF_control_points.xlsx", index=True)

In [75]:
QAPF_control_area2

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,788062.538051,4.936585e+06,53.0,T,2.0,52.526539,17.979983,22.933498,monzo granite
7,609204.390926,5.261750e+06,53.0,T,2.0,43.913578,28.900560,23.948371,monzo granite
8,683161.686140,5.345172e+06,53.0,T,2.0,55.002583,18.817920,19.755253,monzo granite
13,566277.769165,5.115580e+06,53.0,T,2.0,42.445293,29.966574,23.523890,monzo granite
16,513561.171031,5.820370e+06,53.0,T,2.0,40.720371,33.121540,20.140040,monzo granite
...,...,...,...,...,...,...,...,...,...
4638,746609.482852,5.293856e+06,53.0,T,2.0,6.645143,62.556135,9.683687,quartz monzodiorite\nquartz monzogabbro
4642,577326.668959,5.042370e+06,53.0,T,2.0,3.695705,49.300256,19.926339,quartz monzodiorite\nquartz monzogabbro
4649,461509.372701,5.662951e+06,53.0,T,2.0,3.945260,59.519575,6.028671,quartz diorite\nquartz gabbro\nquartz anorthosite
4654,620259.749413,5.148324e+06,53.0,T,2.0,0.000000,52.913423,17.411244,monzodiorite monzogabbro


In [77]:
QAPF_control_area2_K.iloc[:, 8].value_counts().to_excel("../_CIPW/CIPW/AREA2_K/QAPF_counts_control_points.xlsx")